In [1]:
# Import our dependencies
import pandas as pd
import numpy as np
import config as creds
import psycopg2
from sqlalchemy import create_engine

In [2]:
# Import our input dataset for county income
income_df = pd.read_csv('Resources\CAINC1__ALL_AREAS_1969_2021.csv')
income_df.head()

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,1969,1970,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,"""00000""",United States,,CAINC1,1,...,Personal income (thousands of dollars),Thousands of dollars,791229000,855525000,...,14003346000,14189228000,14969527000,15681233000,16092713000,16837337000,17671054000,18575467000,19812171000,21288709000
1,"""00000""",United States,,CAINC1,2,...,Population (persons) 1/,Number of persons,201298000,203798722,...,314344331,316735375,319270047,321829327,324367742,326623063,328542157,330233102,331501080,331893745
2,"""00000""",United States,,CAINC1,3,...,Per capita personal income (dollars) 2/,Dollars,3931,4198,...,44548,44798,46887,48725,49613,51550,53786,56250,59765,64143
3,"""01000""",Alabama,5,CAINC1,1,...,Personal income (thousands of dollars),Thousands of dollars,9737715,10628318,...,173361655,175185262,181078773,189115437,192363456,199191774,206712014,216587513,232040332,250828813
4,"""01000""",Alabama,5,CAINC1,2,...,Population (persons) 1/,Number of persons,3440000,3449846,...,4839261,4864399,4886793,4908162,4930595,4952202,4976395,5003418,5024803,5039877


In [3]:
income_df.dtypes

GeoFIPS      object
GeoName      object
Region       object
TableName    object
LineCode      int64
              ...  
2017         object
2018         object
2019         object
2020         object
2021         object
Length: 61, dtype: object

In [4]:
income_df.Description.value_counts()

Personal income (thousands of dollars)     3200
Population (persons) 1/                    3200
Per capita personal income (dollars) 2/    3200
Name: Description, dtype: int64

In [5]:
# We are keeping columns GeoName, Description, Units, 2019, 2020, 2021
# dropping irrelevant columns
income_df = income_df[['GeoName', 'Description','Unit', '2019','2020', '2021']]
income_df


,GeoName,Description,Unit,2019,2020,2021
0,United States,Personal income (thousands of dollars),Thousands of dollars,18575467000,19812171000,21288709000
1,United States,Population (persons) 1/,Number of persons,330233102,331501080,331893745
2,United States,Per capita personal income (dollars) 2/,Dollars,56250,59765,64143
3,Alabama,Personal income (thousands of dollars),Thousands of dollars,216587513,232040332,250828813
4,Alabama,Population (persons) 1/,Number of persons,5003418,5024803,5039877
...,...,...,...,...,...,...
9595,Rocky Mountain,Population (persons) 1/,Number of persons,12427477,12577224,12734041
9596,Rocky Mountain,Per capita personal income (dollars) 2/,Dollars,55312,58648,62879
9597,Far West,Personal income (thousands of dollars),Thousands of dollars,3566194765,3861508662,4164036360
9598,Far West,Population (persons) 1/,Number of persons,56663461,56758490,56540900


In [6]:
income_df.Unit.value_counts()

Thousands of dollars    3200
Number of persons       3200
Dollars                 3200
Name: Unit, dtype: int64

In [7]:
# dropping irrelevant rows as we only need data where description="Personal income (thousands of dollars)"
income_df = income_df[income_df['Unit'] == "Thousands of dollars"]
income_df

,GeoName,Description,Unit,2019,2020,2021
0,United States,Personal income (thousands of dollars),Thousands of dollars,18575467000,19812171000,21288709000
3,Alabama,Personal income (thousands of dollars),Thousands of dollars,216587513,232040332,250828813
6,"Autauga, AL",Personal income (thousands of dollars),Thousands of dollars,2495580,2664063,2857062
9,"Baldwin, AL",Personal income (thousands of dollars),Thousands of dollars,10986182,11971269,13079505
12,"Barbour, AL",Personal income (thousands of dollars),Thousands of dollars,878887,934683,1009235
...,...,...,...,...,...,...
9585,Plains,Personal income (thousands of dollars),Thousands of dollars,1148499385,1223899119,1302890527
9588,Southeast,Personal income (thousands of dollars),Thousands of dollars,4240360863,4518896149,4899059768
9591,Southwest,Personal income (thousands of dollars),Thousands of dollars,2184468544,2299297951,2492632840
9594,Rocky Mountain,Personal income (thousands of dollars),Thousands of dollars,687390595,737628888,800697862


In [8]:
# dropping rows where GeoName = United States
income_df.drop(income_df[income_df['GeoName'] == "United States"].index, inplace=True)
income_df

C:\Users\MEGHA\anaconda3\envs\mlenv\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,GeoName,Description,Unit,2019,2020,2021
3,Alabama,Personal income (thousands of dollars),Thousands of dollars,216587513,232040332,250828813
6,"Autauga, AL",Personal income (thousands of dollars),Thousands of dollars,2495580,2664063,2857062
9,"Baldwin, AL",Personal income (thousands of dollars),Thousands of dollars,10986182,11971269,13079505
12,"Barbour, AL",Personal income (thousands of dollars),Thousands of dollars,878887,934683,1009235
15,"Bibb, AL",Personal income (thousands of dollars),Thousands of dollars,712472,768870,829213
...,...,...,...,...,...,...
9585,Plains,Personal income (thousands of dollars),Thousands of dollars,1148499385,1223899119,1302890527
9588,Southeast,Personal income (thousands of dollars),Thousands of dollars,4240360863,4518896149,4899059768
9591,Southwest,Personal income (thousands of dollars),Thousands of dollars,2184468544,2299297951,2492632840
9594,Rocky Mountain,Personal income (thousands of dollars),Thousands of dollars,687390595,737628888,800697862


In [9]:
# drop columns Unit, Description
income_df = income_df[['GeoName','2019','2020','2021']]
income_df

,GeoName,2019,2020,2021
3,Alabama,216587513,232040332,250828813
6,"Autauga, AL",2495580,2664063,2857062
9,"Baldwin, AL",10986182,11971269,13079505
12,"Barbour, AL",878887,934683,1009235
15,"Bibb, AL",712472,768870,829213
...,...,...,...,...
9585,Plains,1148499385,1223899119,1302890527
9588,Southeast,4240360863,4518896149,4899059768
9591,Southwest,2184468544,2299297951,2492632840
9594,Rocky Mountain,687390595,737628888,800697862


In [10]:
income_df.dtypes

GeoName    object
2019       object
2020       object
2021       object
dtype: object

In [11]:
#converting income columns to numeric
income_df['2019'] = pd.to_numeric(income_df['2019'], errors='coerce')
income_df

C:\Users\MEGHA\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,GeoName,2019,2020,2021
3,Alabama,2.165875e+08,232040332,250828813
6,"Autauga, AL",2.495580e+06,2664063,2857062
9,"Baldwin, AL",1.098618e+07,11971269,13079505
12,"Barbour, AL",8.788870e+05,934683,1009235
15,"Bibb, AL",7.124720e+05,768870,829213
...,...,...,...,...
9585,Plains,1.148499e+09,1223899119,1302890527
9588,Southeast,4.240361e+09,4518896149,4899059768
9591,Southwest,2.184469e+09,2299297951,2492632840
9594,Rocky Mountain,6.873906e+08,737628888,800697862


In [12]:
income_df['2020'] = pd.to_numeric(income_df['2020'], errors='coerce')
income_df

C:\Users\MEGHA\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,GeoName,2019,2020,2021
3,Alabama,2.165875e+08,2.320403e+08,250828813
6,"Autauga, AL",2.495580e+06,2.664063e+06,2857062
9,"Baldwin, AL",1.098618e+07,1.197127e+07,13079505
12,"Barbour, AL",8.788870e+05,9.346830e+05,1009235
15,"Bibb, AL",7.124720e+05,7.688700e+05,829213
...,...,...,...,...
9585,Plains,1.148499e+09,1.223899e+09,1302890527
9588,Southeast,4.240361e+09,4.518896e+09,4899059768
9591,Southwest,2.184469e+09,2.299298e+09,2492632840
9594,Rocky Mountain,6.873906e+08,7.376289e+08,800697862


In [13]:
income_df['2021'] = pd.to_numeric(income_df['2021'], errors='coerce')
income_df

C:\Users\MEGHA\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,GeoName,2019,2020,2021
3,Alabama,2.165875e+08,2.320403e+08,2.508288e+08
6,"Autauga, AL",2.495580e+06,2.664063e+06,2.857062e+06
9,"Baldwin, AL",1.098618e+07,1.197127e+07,1.307950e+07
12,"Barbour, AL",8.788870e+05,9.346830e+05,1.009235e+06
15,"Bibb, AL",7.124720e+05,7.688700e+05,8.292130e+05
...,...,...,...,...
9585,Plains,1.148499e+09,1.223899e+09,1.302891e+09
9588,Southeast,4.240361e+09,4.518896e+09,4.899060e+09
9591,Southwest,2.184469e+09,2.299298e+09,2.492633e+09
9594,Rocky Mountain,6.873906e+08,7.376289e+08,8.006979e+08


In [14]:
income_df.dtypes

GeoName     object
2019       float64
2020       float64
2021       float64
dtype: object

In [15]:
# dropping rows where three year columns have NaN
income_df = income_df.dropna(subset=['GeoName'])
income_df = income_df.dropna(thresh=2)
income_df

,GeoName,2019,2020,2021
3,Alabama,2.165875e+08,2.320403e+08,2.508288e+08
6,"Autauga, AL",2.495580e+06,2.664063e+06,2.857062e+06
9,"Baldwin, AL",1.098618e+07,1.197127e+07,1.307950e+07
12,"Barbour, AL",8.788870e+05,9.346830e+05,1.009235e+06
15,"Bibb, AL",7.124720e+05,7.688700e+05,8.292130e+05
...,...,...,...,...
9585,Plains,1.148499e+09,1.223899e+09,1.302891e+09
9588,Southeast,4.240361e+09,4.518896e+09,4.899060e+09
9591,Southwest,2.184469e+09,2.299298e+09,2.492633e+09
9594,Rocky Mountain,6.873906e+08,7.376289e+08,8.006979e+08


In [16]:
# creating new column
income_df["statecode_of_county"] = ""
# changing the order of columns
income_df = income_df[['statecode_of_county','GeoName','2019','2020','2021']]

In [17]:
# Change the names of the columns
income_df.columns = ['statecode_of_county','county', 'income_yr_2019', 'income_yr_2020', 'income_yr_2021']

In [18]:
#Connecting to database
db_string = f"postgresql://{creds.PGUSER}:{creds.PGPASSWORD}@{creds.PGHOST}:5432/{creds.PGDATABASE}"
engine = create_engine(db_string)
conn = engine.connect()

In [19]:
#insert the dataframe to postgress table stripping the index
income_df.to_sql(name='household_income', con = conn, if_exists='append', index=False)

In [20]:
# TO-DO
# separate county and statecode and populate columns accordingly in database after creating county table